# MIMIC-IV Clinical Database Demo Data exploration using SQL

#### In this notebook, we will explore this dataset using SQL


## Data exploration 

The Medical Information Mart for Intensive Care (MIMIC)-IV database is comprised
of deidentified electronic health records for patients admitted to the Beth Israel
Deaconess Medical Center. Here, we are looking at an openly-available demo of MIMIC-IV containing a subset
of 100 patients. The dataset includes similar content to MIMIC-IV, but excludes
free-text clinical notes. 
For details on the data, see the MIMIC-IV project on PhysioNet:
https://doi.org/10.13026/07hj-2a80

In [2]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

df1 = pd.read_csv('diagnoses_icd.csv.gz', compression='gzip')
df2 = pd.read_csv('d_icd_diagnoses.csv.gz', compression='gzip')
df3 = pd.read_csv('admissions.csv.gz', compression='gzip')
int = pd.merge(df1, df2, on=['icd_code', 'icd_version'], how='left')
int2 = pd.merge(int, df3, on=['subject_id', 'hadm_id'], how='left')
df6 = pd.read_csv('patients.csv.gz', compression='gzip')
df = pd.merge(int2, df6, on=['subject_id'], how='left')
df = df.drop(columns=['edregtime', 'edouttime'])
df['admitlen'] = (pd.to_datetime(df['dischtime']) - pd.to_datetime(df['admittime']))/ np.timedelta64(1, 'D')
df4 = pd.read_csv('pharmacy.csv.gz', compression='gzip')
df4['ttt'] = [1 if x == 'Heparin' else 0 for x in df4['medication']]
df['admittime'] = pd.to_datetime(df['admittime']) 
df = pd.merge(df, df4[['subject_id', 'hadm_id','ttt']], on=['subject_id', 'hadm_id'], how='left')
df['gender2'] = [1 if x == 'F' else 0 for x in df['gender']]
df = df.dropna(subset=['ttt'])
df = df.dropna(subset=['admittime'])
df.head()

# Create SQLite DB file
engine = create_engine("sqlite:///example.db")

# Write CSV into a SQL table
df.to_sql("df", engine, if_exists="replace", index=False)

# provided by jupysql
%load_ext sql
%sql sqlite:///example.db

Connecting to 'sqlite:///example.db'

1. First, let's have a **look at our data**.

In [4]:
%%sql
SELECT * FROM df LIMIT 10;

Running query in 'sqlite:///example.db'

subject_id,hadm_id,seq_num,icd_code,icd_version,long_title,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,admitlen,ttt,gender2
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0
10035185,22580999,3,4139,9,Other and unspecified angina pectoris,2120-05-12 12:53:00.000000,2120-05-17 16:00:00,None,URGENT,P41R5N,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,0,M,70,2120,2014 - 2016,None,5.129861111111111,0.0,0


2. **How many records** do we have?

In [19]:
%%sql
SELECT COUNT(*) FROM df;	

Running query in 'sqlite:///example.db'

COUNT(*)
322343


3. We want to know the **different types of admission**

In [17]:
%%sql
SELECT DISTINCT admission_type
  FROM df

Running query in 'sqlite:///example.db'

admission_type
URGENT
EW EMER.
ELECTIVE
SURGICAL SAME DAY ADMISSION
DIRECT EMER.
OBSERVATION ADMIT
DIRECT OBSERVATION
EU OBSERVATION
AMBULATORY OBSERVATION


4. We can check if some column had **nulls**

In [68]:
%%sql 
select sum(case when admission_location is null then 1 else 0 end) count_nulls
     , count(admission_location) count_not_nulls 
  from df;

Running query in 'sqlite:///example.db'

count_nulls,count_not_nulls
0,322343


5. What was the **minimum length of stay?**

In [22]:
%%sql
SELECT MIN(admitlen) as Min_stay from df

Running query in 'sqlite:///example.db'

Min_stay
0.49375


6. We want to know **how many patients had a diabetes related icd**

In [6]:
%%sql
SELECT
    CASE WHEN LOWER(long_title) LIKE '%diabetes%' THEN 1 ELSE 0 END AS diab,
    COUNT(DISTINCT subject_id) AS count
FROM df
GROUP BY diab
HAVING diab =1

Running query in 'sqlite:///example.db'

diab,count
1,35


7. We can look **how many patients had each type of insurance**.

In [23]:
%%sql
SELECT
  insurance,
  COUNT(DISTINCT subject_id) AS patient_count
FROM df
GROUP BY insurance
ORDER BY patient_count DESC;

Running query in 'sqlite:///example.db'

insurance,patient_count
Other,57
Medicare,38
Medicaid,10


8. We want to know the **5 most frequent ICD diagnosis per patient**

In [7]:
%%sql
SELECT long_title,
       COUNT(*) AS count
FROM (
  SELECT DISTINCT subject_id, long_title
  FROM df
) t
GROUP BY long_title
ORDER BY count DESC
LIMIT 5

Running query in 'sqlite:///example.db'

long_title,count
Unspecified essential hypertension,40
Other and unspecified hyperlipidemia,34
"Anemia, unspecified",25
"Acute kidney failure, unspecified",24
"Urinary tract infection, site not specified",21


9. Lets count the **number of patients who had both a diagnostic of diabetes and heart failure in the same admission**

In [8]:
%%sql
SELECT COUNT(*) AS patient_count
FROM (
    SELECT
      a.subject_id,
      a.hadm_id
    FROM df a
    JOIN df b
      ON a.subject_id = b.subject_id
     AND a.hadm_id = b.hadm_id
    WHERE LOWER(a.long_title) LIKE '%diabetes%'
      AND LOWER(b.long_title) LIKE '%heart failure%'
    GROUP BY a.subject_id, a.hadm_id
) t;

Running query in 'sqlite:///example.db'

patient_count
35


10. We use a **Window function** to add columns showing:
* the **average admission time over all admissions where the same subject_id had that icd_code**
* the **average admission time over all admissions where ALL subject_id had that icd_code**

In [20]:
%%sql
SELECT
  subject_id,
  hadm_id,
  icd_code,
  admitlen,
  -- Average LOS for THIS patient for THIS ICD
  AVG(admitlen) OVER (
    PARTITION BY subject_id, icd_code
  ) AS avg_admitlen_patient_icd,
  -- Average LOS across ALL patients for THIS ICD
  AVG(admitlen) OVER (
    PARTITION BY icd_code
  ) AS avg_admitlen_allpatients_icd
FROM df
ORDER BY avg_admitlen_allpatients_icd DESC
LIMIT 10;

Running query in 'sqlite:///example.db'

subject_id,hadm_id,icd_code,admitlen,avg_admitlen_patient_icd,avg_admitlen_allpatients_icd
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778
10020740,23831430,29520,44.92777777777778,44.92777777777778,44.92777777777778


11. We can look at the **number of deaths per patient per race**

In [39]:
%%sql
SELECT t.race,
       ROUND(100*t.countd/t.countid,1) AS avg_death
FROM (
    SELECT
      COUNT(a.subject_id) AS countid,
      SUM(a.hospital_expire_flag) AS countd,
      a.race
     FROM df a
    GROUP BY a.race
) t
ORDER BY avg_death DESC

Running query in 'sqlite:///example.db'

race,avg_death
PATIENT DECLINED TO ANSWER,82.0
UNKNOWN,39.0
BLACK/AFRICAN AMERICAN,15.0
WHITE,14.0
BLACK/CAPE VERDEAN,0.0
HISPANIC OR LATINO,0.0
HISPANIC/LATINO - CUBAN,0.0
HISPANIC/LATINO - PUERTO RICAN,0.0
HISPANIC/LATINO - SALVADORAN,0.0
OTHER,0.0


12. We want to **list all unique icd diagnosis for each patient**

In [55]:
%%sql
SELECT
    subject_id,
    GROUP_CONCAT(long_title, ', ') AS icd
FROM (
    SELECT DISTINCT subject_id, long_title
    FROM df
)
GROUP BY subject_id;

Running query in 'sqlite:///example.db'

+------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

13. We want to know **for each patient if the patient should be rescucitated** or not

In [59]:
%%sql
SELECT
    DISTINCT subject_id,
    CASE WHEN LOWER(long_title) LIKE '%do not resuscitate%' THEN "do not resc" ELSE "do resc" END AS rescucitation
FROM df
GROUP BY subject_id 

Running query in 'sqlite:///example.db'

subject_id,rescucitation
10000032,do resc
10001217,do resc
10001725,do resc
10002428,do resc
10002495,do resc
10002930,do resc
10003046,do resc
10003400,do resc
10004235,do resc
10004422,do resc
